In [1]:
import autosklearn.classification
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import sklearn.metrics

In [2]:
dataset = pd.read_csv('../Datasets/Diabetes/diabetic_data.csv')

In [3]:
dataset.replace('?', np.nan, inplace=True)

In [4]:
dataset.drop(['weight','payer_code','medical_specialty'],axis=1,inplace=True)

In [5]:
dataset.dropna(inplace=True)

In [6]:
dataset.drop(['encounter_id','patient_nbr','admission_type_id','discharge_disposition_id','admission_source_id'],axis=1,inplace=True)

In [7]:
dataset

,race,gender,age,time_in_hospital,num_lab_procedures,num_procedures,num_medications,number_outpatient,number_emergency,number_inpatient,...,citoglipton,insulin,glyburide-metformin,glipizide-metformin,glimepiride-pioglitazone,metformin-rosiglitazone,metformin-pioglitazone,change,diabetesMed,readmitted
1,Caucasian,Female,[10-20),3,59,0,18,0,0,0,...,No,Up,No,No,No,No,No,Ch,Yes,>30
2,AfricanAmerican,Female,[20-30),2,11,5,13,2,0,1,...,No,No,No,No,No,No,No,No,Yes,NO
3,Caucasian,Male,[30-40),2,44,1,16,0,0,0,...,No,Up,No,No,No,No,No,Ch,Yes,NO
4,Caucasian,Male,[40-50),1,51,0,8,0,0,0,...,No,Steady,No,No,No,No,No,Ch,Yes,NO
5,Caucasian,Male,[50-60),3,31,6,16,0,0,0,...,No,Steady,No,No,No,No,No,No,Yes,>30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
101761,AfricanAmerican,Male,[70-80),3,51,0,16,0,0,0,...,No,Down,No,No,No,No,No,Ch,Yes,>30
101762,AfricanAmerican,Female,[80-90),5,33,3,18,0,0,1,...,No,Steady,No,No,No,No,No,No,Yes,NO
101763,Caucasian,Male,[70-80),1,53,0,9,1,0,0,...,No,Down,No,No,No,No,No,Ch,Yes,NO
101764,Caucasian,Female,[80-90),10,45,2,21,0,0,1,...,No,Up,No,No,No,No,No,Ch,Yes,NO


In [8]:
dataset.loc[dataset.age== '[0-10)','age'] = 0;
dataset.loc[dataset.age== '[10-20)','age'] = 10;
dataset.loc[dataset.age== '[20-30)','age'] = 20;
dataset.loc[dataset.age== '[30-40)','age'] = 30;
dataset.loc[dataset.age== '[40-50)','age'] = 40;
dataset.loc[dataset.age== '[50-60)','age'] = 50;
dataset.loc[dataset.age== '[60-70)','age'] = 60;
dataset.loc[dataset.age== '[70-80)','age'] = 70;
dataset.loc[dataset.age== '[80-90)','age'] = 80;
dataset.loc[dataset.age== '[90-100)','age'] = 90;
dataset.age = dataset.age.astype(np.int32)


dataset.loc[dataset.max_glu_serum== 'None','max_glu_serum'] = 0;
dataset.loc[dataset.max_glu_serum== 'Norm','max_glu_serum'] = 100;
dataset.loc[dataset.max_glu_serum== '>200','max_glu_serum'] = 200;
dataset.loc[dataset.max_glu_serum== '>300','max_glu_serum'] = 300;
dataset.max_glu_serum = dataset.max_glu_serum.astype(np.int32)


dataset.loc[dataset.A1Cresult== 'None','A1Cresult'] = 0;
dataset.loc[dataset.A1Cresult== 'Norm','A1Cresult'] = 5;
dataset.loc[dataset.A1Cresult== '>7','A1Cresult'] = 7;
dataset.loc[dataset.A1Cresult== '>8','A1Cresult'] = 8;
dataset.A1Cresult = dataset.A1Cresult.astype(np.int32)


dataset.loc[dataset.change== 'No','change'] = 0;
dataset.loc[dataset.change== 'Ch','change'] = 1;
dataset.change = dataset.change.astype(np.int8)



dataset.loc[dataset.diabetesMed== 'No','diabetesMed'] = 0;
dataset.loc[dataset.diabetesMed== 'Yes','diabetesMed'] = 1;
dataset.diabetesMed = dataset.diabetesMed.astype(np.int8)


medications = ["metformin", "repaglinide", "nateglinide", "chlorpropamide", "glimepiride", "acetohexamide", "glipizide", "glyburide", "tolbutamide", "pioglitazone", "rosiglitazone", "acarbose", "miglitol", "troglitazone", "tolazamide", "examide", "citoglipton", "insulin", "glyburide-metformin", "glipizide-metformin", "glimepiride-pioglitazone", "metformin-rosiglitazone", "metformin-pioglitazone"]

for med in medications:
    dataset.loc[dataset[med] == 'No', med] = -20;
    dataset.loc[dataset[med] == 'Down', med] = -10;
    dataset.loc[dataset[med] == 'Steady', med] = 0;
    dataset.loc[dataset[med] == 'Up', med] = 10;
    dataset[med] = dataset[med].astype(np.int32)
    

categoricals = ['race', 'gender', 'diag_1', 'diag_2', 'diag_3']



for c in categoricals:
    dataset[c] = pd.Categorical(dataset[c]).codes

In [9]:
dataset.loc[dataset.readmitted != 'NO','readmitted'] = 0
dataset.loc[dataset.readmitted == 'NO','readmitted'] = 1

In [10]:
dataset = dataset.astype(np.int8)

In [11]:
X, y = dataset.drop('readmitted', axis=1), dataset['readmitted']
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, stratify=y)

# AUTOML

In [39]:
automl = autosklearn.classification.AutoSklearnClassifier(time_left_for_this_task=360)
automl.fit(X_train.values, y_train)
y_hat = automl.predict(X_test.values)

[WARNING] [2023-07-02 20:02:48,144:Client-EnsembleBuilder] No runs were available to build an ensemble from


In [40]:
sklearn.metrics.accuracy_score(y_test, y_hat)

0.62919148241821

In [41]:
import pickle
with open('./models/diabetes_automl.pkl', 'wb') as f:
    pickle.dump(automl, f)

In [12]:
import pickle
with open('./models/diabetes_automl.pkl', 'rb') as f:
    automl = pickle.load(f)

len(automl.show_models())
# y_hat = automl.predict(X_test.values)
# sklearn.metrics.accuracy_score(y_test, y_hat)

8

# RANDOM FOREST

In [16]:
import sklearn.ensemble

model = sklearn.ensemble.RandomForestClassifier(n_estimators=512, n_jobs=5, random_state=42)
model.fit(X_train, y_train)
y_hat = model.predict(X_test)

In [17]:
sklearn.metrics.accuracy_score(y_test, y_hat)

0.6216447744146202

In [18]:
with open('./models/diabetes_random_forest.pkl', 'wb') as f:
    pickle.dump(model, f)

# SVC

In [15]:
import sklearn.svm
import sklearn.model_selection

model = sklearn.svm.SVC(random_state=42)
params = {'C': [0.1, 1, 10]}
gridSearch = sklearn.model_selection.GridSearchCV(model, param_grid=params, cv=2, n_jobs=5, verbose=3)
gridSearch.fit(X_train, y_train)

Fitting 2 folds for each of 3 candidates, totalling 6 fits


GridSearchCV(cv=2, estimator=SVC(random_state=42), n_jobs=5,
             param_grid={'C': [0.1, 1, 10]}, verbose=3)

In [16]:
y_hat = gridSearch.predict(X_test)
sklearn.metrics.accuracy_score(y_test, y_hat)

0.584155992494085

# LIME

In [14]:
import lime
import lime.lime_tabular
import tqdm 

In [15]:
continuous_features = ['time_in_hospital', 'num_lab_procedures', 'num_procedures', 'num_medications', 'number_outpatient', 'number_emergency', 'number_inpatient', 'diag_1', 'diag_2', 'diag_3', 'number_diagnoses']
categorical_features = X_train.columns.drop(continuous_features).tolist()
explainer = lime.lime_tabular.LimeTabularExplainer(X_train.values, feature_names=X_train.columns.tolist(), class_names=['Readmitted', 'Not Readmitted'], categorical_features=categorical_features, discretize_continuous=True)

In [33]:
frac = 0.02
X_test_sample, _, y_test_sample, _ = train_test_split(X_test, y_test, train_size=frac, random_state=42, stratify=y_test)

test_x = X_test_sample.values
test_y = y_test_sample.values

In [22]:
exp_fn = lambda i: explainer.explain_instance(test_x[i], automl.predict_proba, num_features=len(X_test.columns))
def exp_fn_blk(xtest, exp_fn):
    exp1 = []
    for i in tqdm.tqdm(range(len(xtest))):
        exp = exp_fn(i)
        exp1.append(exp.as_map()[exp.available_labels()[0]])
    return np.array(exp1)
exp_fn_wrap = lambda x: np.array(exp_fn_blk(x, exp_fn))

In [23]:
import metrics

In [18]:
exp1 = exp_fn_wrap(test_x)
exp2 = exp_fn_wrap(test_x)

100%|██████████| 490/490 [54:52<00:00,  6.72s/it]    


In [19]:
np.save('./explanations/diabetes1.npy', exp1)
np.save('./explanations/diabetes2.npy', exp2)

In [24]:
exp1 = np.load('./explanations/diabetes1.npy')
exp2 = np.load('./explanations/diabetes2.npy')

In [25]:
def enc_exp(exp, feature_num):
    enc_exp = np.zeros((len(exp),feature_num))
    for i in range(len(exp)):
        for j in range(len(exp[i])):
            enc_exp[i][int(exp[i,j,0])] = exp[i,j,1]
    return enc_exp

In [26]:
i = metrics.calc_identity(exp1, exp2)
s = metrics.calc_separability(exp1)
enc1 = enc_exp(exp1, len(X_test.columns))
sb = metrics.calc_stability(enc1, test_y)

/home/krish/anaconda3/envs/interpret/lib/python3.7/site-packages/sklearn/cluster/_kmeans.py:984: RuntimeWarning: Explicit initial center position passed: performing only one init in KMeans instead of n_init=10.
  self._check_params(X)


In [27]:
i, s, sb

((100.0, 0, 490), (0, 490, 240100, 0.0), (192, 490))

In [28]:
X_test_norm = metrics.normalize_test(X_train, X_test_sample)
sim = metrics.calc_similarity(exp1, X_test_norm)

In [29]:
sim

0.3235792449368628

In [25]:
list_monotonicity = []
list_non_sensitivity = []
list_effective_complexity = []

for i in tqdm.tqdm(range(len(test_x))):
    atr = exp1[i]
    sorted_atr = [j for i,j in atr]
    sorted_feat = [i for i,j in atr]
    y = np.zeros(2, dtype=int)
    np.put(y, test_y[i], 1)
    example = metrics.FeatureAttribution(automl, test_x[i], y, sorted_atr)
    list_monotonicity.append(example.monotonicity())
    list_non_sensitivity.append(example.non_sensitivity())
    list_effective_complexity.append(example.effective_complexity(sorted_feat, 0.1))

  0%|          | 0/490 [00:00<?, ?it/s]

100%|██████████| 490/490 [47:31<00:00,  5.82s/it]


In [26]:
print(np.mean(list_monotonicity))
print(np.mean(list_non_sensitivity))
print(np.mean(list_effective_complexity))

print(np.median(list_monotonicity))
print(np.median(list_non_sensitivity))
print(np.median(list_effective_complexity))

0.017664565003795714
8.944897959183674
0.0
0.0030237643705725574
9.0
0.0


In [30]:
trust = metrics.calc_trust_score(automl, test_x, exp1, 3, X_train.columns.tolist())

100%|██████████| 490/490 [3:24:16<00:00, 25.01s/it]    


In [31]:
trust

0.21428571428571427

# LIME Global

In [17]:
from lime import submodular_pick
import time

start_time = time.time()
exp1 = submodular_pick.SubmodularPick(explainer, X_test_sample.values, automl.predict_proba, sample_size=200, num_features=len(X_test_sample.columns), num_exps_desired=5)
print("--- %s seconds ---" % (time.time() - start_time))

start_time = time.time()
exp2 = submodular_pick.SubmodularPick(explainer, X_test_sample.values, automl.predict_proba, sample_size=200, num_features=len(X_test_sample.columns), num_exps_desired=5)
print("--- %s seconds ---" % (time.time() - start_time))

--- 538.3214120864868 seconds ---
--- 528.7014925479889 seconds ---


In [18]:
import pickle
with open('./explanations/diabetes_lime_global1.pkl', 'wb') as f:
    pickle.dump(exp1, f)

with open('./explanations/diabetes_lime_global2.pkl', 'wb') as f:
    pickle.dump(exp2, f)

In [13]:
import pickle
with open('./explanations/diabetes_lime_global1.pkl', 'rb') as f:
    exp1 = pickle.load(f)

with open('./explanations/diabetes_lime_global2.pkl', 'rb') as f:
    exp2 = pickle.load(f)

In [14]:
def get_feature_imp(sp_obj):
    W_pick=pd.DataFrame([dict(this.as_list(this.available_labels()[0])) for this in sp_obj.sp_explanations]).fillna(0)
    W_pick['prediction'] = [this.available_labels()[0] for this in sp_obj.sp_explanations]
    W=pd.DataFrame([dict(this.as_list(this.available_labels()[0])) for this in sp_obj.explanations]).fillna(0)
    W['prediction'] = [this.available_labels()[0] for this in sp_obj.explanations]
    np.abs(W.drop("prediction", axis=1)).mean(axis=0).sort_values(ascending=False).head(25).sort_values(ascending=True)
    grped_coeff = W.groupby("prediction").mean()
    grped_coeff = grped_coeff.T
    return grped_coeff[0].values

In [15]:
feat_imp1 = get_feature_imp(exp1)
feat_imp2 = get_feature_imp(exp2)

In [16]:
feat_imp1

array([-5.94970596e-02,  3.90620310e-02,  2.32662770e-02,  1.40797886e-03,
        2.52952134e-03, -1.23929943e-04, -2.95757066e-02, -8.71038688e-03,
        8.27913931e-04, -5.38385658e-03,  9.35783612e-03, -1.84607045e-02,
        1.02286008e-02, -2.47564839e-03,  1.88735530e-02, -1.36632873e-03,
       -2.93947051e-03,  2.11718548e-03,  4.23474778e-03,  2.43861870e-04,
        9.71351806e-04,  6.58040173e-05, -1.75528740e-04,  2.05163934e-03,
       -4.69336464e-04, -2.01329118e-03,  1.64547431e-03, -1.23655440e-04,
       -6.98920891e-04,  6.43259701e-04, -5.06287048e-04,  5.29642676e-04,
        2.61169942e-03,  0.00000000e+00,  0.00000000e+00, -6.50211980e-04,
       -6.58310208e-04,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00, -8.29703262e-02,  2.36990653e-02,  4.83198793e-03,
       -6.85357902e-03, -3.85530412e-04,  8.01615595e-04, -1.09195502e-03,
       -1.26165962e-04,  2.46400685e-03,  6.55483353e-04,  2.45783401e-05,
        2.87016948e-04,  

In [17]:
def global_identity(feat_imp1, feat_imp2):
    sum = 0
    for i in range(len(feat_imp1)):
        if(feat_imp1[i] == feat_imp2[i]):
            sum += 1
    return sum/len(feat_imp1)

In [18]:
i = global_identity(feat_imp1, feat_imp2)
i

0.05747126436781609

In [19]:
def normal_fi(feat_imp):
    return np.abs(feat_imp) / np.sum(np.abs(feat_imp))

In [20]:
normal_feat_imp = normal_fi(feat_imp1 + 1e-9)

In [21]:
#Entropy Ratio
Ser = np.sum(normal_feat_imp*np.log(normal_feat_imp))/np.log(1/len(normal_feat_imp))

# Kullback-Leibler Divergence
Skl = np.sum(normal_feat_imp*np.log(normal_feat_imp/(1/len(normal_feat_imp))))

In [22]:
def calc_gini(pfi):
    sum = 0
    for i in range(len(pfi)):
        sum_curr = 0
        for j in range(len(pfi)):
            sum_curr += np.abs(pfi[i]-pfi[j])
        sum += sum_curr
    
    return sum/(2*len(pfi)**2)*(np.sum(pfi)/len(pfi))

In [23]:
Sg = calc_gini(normal_feat_imp)

In [24]:
Ser, Skl, Sg

(0.7366600451611253, 1.176052042281062, 0.00010118636840223408)

In [25]:
def calc_alpha_fi(normal_pfi, alpha):
    j_inst = 0
    sum = 0
    for i in range(len(normal_pfi)-1, -1, -1):
        sum += normal_pfi[i]
        if sum<=alpha:
            j_inst = i
        else:
            break
    return 1- (j_inst/len(normal_pfi))

In [26]:
calc_alpha_fi(normal_feat_imp, 0.8)

0.9195402298850575

In [27]:
def get_feature_imp_all(sp_obj):
    W_pick=pd.DataFrame([dict(this.as_list(this.available_labels()[0])) for this in sp_obj.sp_explanations]).fillna(0)
    W_pick['prediction'] = [this.available_labels()[0] for this in sp_obj.sp_explanations]
    W=pd.DataFrame([dict(this.as_list(this.available_labels()[0])) for this in sp_obj.explanations]).fillna(0)
    W['prediction'] = [this.available_labels()[0] for this in sp_obj.explanations]
    np.abs(W.drop("prediction", axis=1)).mean(axis=0).sort_values(ascending=False).head(25).sort_values(ascending=True)
    grped_coeff = W.groupby("prediction").mean()
    grped_coeff = grped_coeff.T
    return grped_coeff

In [28]:
class1_feat_imp, class2_feat_imp = get_feature_imp_all(exp1)[0].values, get_feature_imp_all(exp1)[1].values
normal_class1_fi, normal_class2_fi = normal_fi(class1_feat_imp), normal_fi(class2_feat_imp)

In [29]:
np.linalg.norm(normal_class1_fi - normal_class2_fi, ord=2)

0.1874718408534028

In [30]:
def get_limits_and_names(idxs):
    data = {}
    for idx in idxs:
        
        data[idx] = {}
        col_name = ""
        for col in X_train.columns:
            if str(col) in idx:
                col_name = col
                break
        data[idx]['col_name'] = col_name
        
        split_lt = idx.split("<=")
        if len(split_lt) > 1:
            for i in range(len(split_lt)):
                split_lt[i] = split_lt[i].strip()
                try:
                    split_lt[i] = float(split_lt[i])
                    data[idx]['upper'] = split_lt[i]
                except:
                    pass
            split_lt2 = split_lt[0].split("<")
            if len(split_lt2) > 1:
                for i in range(len(split_lt2)):
                    split_lt2[i] = split_lt2[i].strip()
                    try:
                        split_lt2[i] = float(split_lt2[i])
                        data[idx]['lower'] = split_lt2[i]
                    except:
                        pass
        split_gt = idx.split(">")
        if len(split_gt) > 1:
            for i in range(len(split_gt)):
                split_gt[i] = split_gt[i].strip()
                try:
                    split_gt[i] = float(split_gt[i])
                    data[idx]['lower'] = split_gt[i]
                except:
                    pass
    return data

In [34]:
mean_correctness = 0
feat_scores = get_feature_imp_all(exp1)
feat_scores_index = feat_scores.index.tolist()
exp_feature_name = get_limits_and_names(feat_scores_index)
for i in range(len(X_test_sample)):
    correctness = 0
    for j in range(len(feat_scores_index)):
        inst = 1
        if exp_feature_name[feat_scores_index[j]]['col_name']:
            if 'upper' in exp_feature_name[feat_scores_index[j]].keys():
                if X_test_sample.iloc[i][exp_feature_name[feat_scores_index[j]]['col_name']] > exp_feature_name[feat_scores_index[j]]['upper']:
                    inst = 0
            if 'lower' in exp_feature_name[feat_scores_index[j]].keys():
                if X_test_sample.iloc[i][exp_feature_name[feat_scores_index[j]]['col_name']] <= exp_feature_name[feat_scores_index[j]]['lower']:
                    inst = 0
                    
        # print(inst, feat_scores.iloc[j][y_test.iloc[i]], feat_scores_index[j], exp_feature_name[feat_scores_index[j]]['col_name'], X_test_sample.iloc[i][exp_feature_name[feat_scores_index[j]]['col_name']])
        correctness += inst * 1
    if correctness < 0:
        correctness = 0
    mean_correctness += correctness/len(feat_scores_index)
print(mean_correctness/len(X_test_sample))

0.46368754398311435


# CIU

In [18]:
from ciu import determine_ciu
import tqdm
import metrics

In [19]:
def enc_exp(exp, feature_num):
    enc_exp = np.zeros((len(exp),feature_num))
    for i in range(len(exp)):
        for j in range(len(exp[i])):
            enc_exp[i][int(exp[i,j,0])] = exp[i,j,1]
    return enc_exp

In [20]:
feat_list = X_train.columns.tolist()

In [21]:
def exp_fn_blk(xtest):
    exp1 = []
    for i in tqdm.tqdm(range(len(xtest))):
        exp = determine_ciu(X_test.iloc[i:i+1], automl.predict_proba, X_train.to_dict('list'), samples = 1000, prediction_index = 1)
        exp_list = [[feat_list.index(i), exp.ci[i]] for i in exp.ci]
        exp1.append(exp_list)
    return np.array(exp1)

In [22]:
exp1 = exp_fn_blk(X_test_sample)
exp2 = exp_fn_blk(X_test_sample)

100%|██████████| 490/490 [42:35<00:00,  5.21s/it]


In [23]:
np.save('./explanations/diabetes_ciu1.npy', exp1)
np.save('./explanations/diabetes_ciu2.npy', exp2)

In [ ]:
exp1 = np.load('./explanations/hiv_ciu1.npy')
exp2 = np.load('./explanations/hiv_ciu2.npy')

In [25]:
i = metrics.calc_identity(exp1, exp2)
s = metrics.calc_separability(exp1)
enc1 = enc_exp(exp1, len(feat_list))
sb = metrics.calc_stability(enc1, y_test_sample)

/home/krish/anaconda3/envs/interpret/lib/python3.7/site-packages/sklearn/cluster/_kmeans.py:984: RuntimeWarning: Explicit initial center position passed: performing only one init in KMeans instead of n_init=10.
  self._check_params(X)


In [26]:
i, s, sb

((100.0, 0, 490), (0, 490, 240100, 0.0), (233, 490))

In [27]:
X_test_norm = metrics.normalize_test(X_train, X_test_sample)
sim = metrics.calc_similarity(exp1, X_test_norm)

In [28]:
sim

0.5912382359091071

In [30]:
list_monotonicity = []
list_non_sensitivity = []
list_effective_complexity = []

for i in tqdm.tqdm(range(len(X_test_sample))):
    atr = exp1[i]
    sorted_atr = [j for i,j in atr]
    sorted_feat = [i for i,j in atr]
    y = np.zeros(2, dtype=int)
    np.put(y, y_test_sample.iloc[i], 1)
    example = metrics.FeatureAttribution(automl, X_test_sample.to_numpy()[i], y, sorted_atr)
    list_monotonicity.append(example.monotonicity())
    list_non_sensitivity.append(example.non_sensitivity())
    list_effective_complexity.append(example.effective_complexity(sorted_feat, 0.1))

100%|██████████| 490/490 [54:35<00:00,  6.69s/it]  


In [31]:
print(np.mean(list_monotonicity))
print(np.mean(list_non_sensitivity))
print(np.mean(list_effective_complexity))

print(np.median(list_monotonicity))
print(np.median(list_non_sensitivity))
print(np.median(list_effective_complexity))

0.015983156317531437
5.995918367346939
0.05714285714285714
0.010337947072485944
6.0
0.0


In [32]:
metrics.calc_trust_score(automl, X_test_sample.to_numpy(), exp1, 3, X_train.columns.to_list())

100%|██████████| 490/490 [2:39:54<00:00, 19.58s/it]  


0.27755102040816326

# RULEFIT

In [33]:
from skrules import SkopeRules
import metrics_rules
import time

In [34]:
clf = SkopeRules(max_depth_duplication=2,
                    n_estimators=512,
                    precision_min=0.3,
                    recall_min=0.1,
                    feature_names=X_train.columns.tolist())

In [35]:
start_time = time.time()
clf.fit(X_train, y_train)
print("--- %s seconds ---" % (time.time() - start_time))

--- 106.79994249343872 seconds ---


In [36]:
start_time = time.time()
top_rules1 = clf.score_top_rules(X_test_sample)
top_rules2 = clf.score_top_rules(X_test_sample)
print("--- %s seconds ---" % (time.time() - start_time))

--- 0.03854656219482422 seconds ---


In [37]:
i = metrics_rules.calc_identity_rules(top_rules1, top_rules2)
print(i)

s = metrics_rules.calc_separability_rules(top_rules1)
print(s)

enc_rules = metrics_rules.exp_enc(clf, top_rules1)
sb = metrics_rules.calc_stability_rules(enc_rules, y_test_sample)
print(sb)

(0.0, 490, 490)
(76788, 490, 240100, 31.98167430237401)
(225, 490)


/home/krish/anaconda3/envs/interpret/lib/python3.7/site-packages/sklearn/cluster/_kmeans.py:984: RuntimeWarning: Explicit initial center position passed: performing only one init in KMeans instead of n_init=10.
  self._check_params(X)


In [38]:
X_test_norm = metrics_rules.normalize_test(X_train, X_test_sample)
sim = metrics_rules.calc_similarity(enc_rules, X_test_norm)
print(sim)

0.9298346247582785


# RULEMATRIX

In [ ]:
categorical_features = X_train.columns.tolist()
continuous_features = X_train.columns.drop(categorical_features).tolist()

In [40]:
import rulematrix
import time
import metrics_rules

In [41]:
is_continuous = [True if i in continuous_features else False for i in X_train.columns.tolist()]
is_categorical = [True if i in categorical_features else False for i in X_train.columns.tolist()]

In [42]:
surrogate = rulematrix.surrogate.rule_surrogate(
    automl.predict,
    X_train,
    sampling_rate=4,
    is_continuous=is_continuous,
    is_categorical=is_categorical,
    seed=42
)

In [43]:
test_x = X_test_sample.to_numpy()

In [44]:
def exp_fn_blk(xtest):
    exp1 = []
    for i in range(len(xtest)):
        queried_rules = np.arange(surrogate.student.n_rules)[surrogate.student.decision_path(test_x[i].reshape(1,-1)).reshape(-1)]
        exp1.append(queried_rules[-1])
    return np.array(exp1)
exp_fn_wrap = lambda x: np.array(exp_fn_blk(x))

In [45]:
start_time = time.time()
exp1 = exp_fn_blk(test_x)
exp2 = exp_fn_blk(test_x)
print("--- %s seconds ---" % (time.time() - start_time))

--- 0.5698792934417725 seconds ---


In [46]:
def enc_exp(exp, n_features):
    enc = []
    for i in range(exp.shape[0]):
        new = np.zeros(n_features)
        for j in surrogate.student.rule_list[exp[i]].clauses:
            new[j.feature_idx] = 1
        enc.append(new)
    return np.array(enc)

In [47]:
enc_exp = enc_exp(exp1, X_train.shape[1])

In [49]:
i = metrics_rules.calc_identity_rules(exp1, exp2)
print(i)

s = metrics_rules.calc_separability_rules(exp1)
print(s)

sb = metrics_rules.calc_stability_rules(enc_exp, y_test_sample)
print(sb)

(0.0, 490, 490)
(13066, 490, 240100, 5.441899208663057)
(197, 490)


/home/krish/anaconda3/envs/interpret/lib/python3.7/site-packages/sklearn/cluster/_kmeans.py:984: RuntimeWarning: Explicit initial center position passed: performing only one init in KMeans instead of n_init=10.
  self._check_params(X)


In [50]:
X_test_norm = metrics_rules.normalize_test(X_train, X_test_sample)
sim = metrics_rules.calc_similarity(enc_exp, X_test_norm)

In [51]:
sim

1.7731358947906135

# ANCHOR Global


In [13]:
from anchor import anchor_tabular
import anchor_utils
import tqdm

In [14]:
explainer = anchor_tabular.AnchorTabularExplainer(
    y_train.unique().tolist(),
    X_train.columns.tolist(),
    X_train.values
)

In [15]:
# Feature Importance using Anchor
def calc_fi(X_test, model, explainer):
    all_exps = []
    for i in tqdm.tqdm(range(len(X_test))):
        exp = explainer.explain_instance(X_test.values[i], model.predict, threshold=0.95)
        all_exps.append(exp.exp_map)
    fi = anchor_utils.greedy_pick_anchor(all_exps, X_test.values, k = len(X_test.columns))
    return fi
        

In [22]:
exp1 = calc_fi(X_test_sample, automl, explainer)
exp2 = calc_fi(X_test_sample, automl, explainer)

100%|██████████| 490/490 [2:14:32<00:00, 16.48s/it]  


0 0.08571428571428572
1 0.15918367346938775
2 0.22857142857142856
3 0.2938775510204082
4 0.33877551020408164
5 0.3836734693877551
6 0.41836734693877553
7 0.4530612244897959
8 0.4816326530612245
9 0.5081632653061224
10 0.5285714285714286
11 0.5428571428571428
12 0.5530612244897959
13 0.563265306122449
14 0.573469387755102
15 0.5836734693877551
16 0.5918367346938775
17 0.6
18 0.6081632653061224
19 0.6163265306122448
20 0.6224489795918368
21 0.6285714285714286
22 0.6346938775510204
23 0.6408163265306123
24 0.6469387755102041
25 0.6530612244897959
26 0.6571428571428571
27 0.6612244897959184
28 0.6653061224489796
29 0.6693877551020408
30 0.673469387755102
31 0.6775510204081633
32 0.6816326530612244
33 0.6857142857142857
34 0.689795918367347
35 0.6938775510204082
36 0.6979591836734694
37 0.7020408163265306
38 0.7061224489795919
39 0.7081632653061225
40 0.710204081632653


100%|██████████| 490/490 [2:15:39<00:00, 16.61s/it]  

0 0.30612244897959184
1 0.40816326530612246
2 0.4775510204081633
3 0.5163265306122449
4 0.5510204081632653
5 0.573469387755102
6 0.5938775510204082
7 0.6122448979591837
8 0.6306122448979592
9 0.6489795918367347
10 0.6591836734693878
11 0.6693877551020408
12 0.6775510204081633
13 0.6857142857142857
14 0.6938775510204082
15 0.7020408163265306
16 0.7081632653061225
17 0.7142857142857143
18 0.7204081632653061
19 0.726530612244898
20 0.7326530612244898
21 0.7387755102040816
22 0.7428571428571429
23 0.746938775510204
24 0.7510204081632653
25 0.7551020408163265
26 0.7591836734693878
27 0.763265306122449
28 0.7673469387755102
29 0.7714285714285715
30 0.7755102040816326
31 0.7795918367346939
32 0.7816326530612245
33 0.7836734693877551
34 0.7857142857142857
35 0.7877551020408163
36 0.789795918367347
37 0.7918367346938775
38 0.7938775510204081
39 0.7959183673469388
40 0.7979591836734694


In [23]:
import pickle
with open('./explanations/diabtetes_anc_global1.pkl', 'wb') as f:
    pickle.dump(exp1, f)

with open('./explanations/diabtetes_anc_global2.pkl', 'wb') as f:
    pickle.dump(exp2, f)

In [17]:
import pickle
with open('./explanations/diabtetes_anc_global1.pkl', 'rb') as f:
    exp1 = pickle.load(f)

with open('./explanations/diabtetes_anc_global2.pkl', 'rb') as f:
    exp2 = pickle.load(f)

In [24]:
X_test_sample_0 = X_test_sample[y_test_sample==0]
X_test_sample_1 = X_test_sample[y_test_sample==1]

exp_0 = calc_fi(X_test_sample_0, automl, explainer)
exp_1 = calc_fi(X_test_sample_1, automl, explainer)

with open('./explanations/diabtetes_anc_global_class0.pkl', 'wb') as f:
    pickle.dump(exp_0, f)

with open('./explanations/diabtetes_anc_global_class1.pkl', 'wb') as f:
    pickle.dump(exp_1, f)

100%|██████████| 228/228 [1:09:03<00:00, 18.17s/it] 


0 0.09210526315789473
1 0.15789473684210525
2 0.20175438596491227
3 0.2412280701754386
4 0.27631578947368424
5 0.30701754385964913
6 0.33771929824561403
7 0.36403508771929827
8 0.38596491228070173
9 0.40789473684210525
10 0.4298245614035088
11 0.4473684210526316
12 0.4649122807017544
13 0.4780701754385965
14 0.49122807017543857
15 0.5043859649122807
16 0.5175438596491229
17 0.5263157894736842
18 0.5350877192982456
19 0.543859649122807
20 0.5526315789473685
21 0.5614035087719298
22 0.5701754385964912
23 0.5789473684210527
24 0.5833333333333334
25 0.5877192982456141
26 0.5921052631578947
27 0.5964912280701754
28 0.6008771929824561
29 0.6052631578947368
30 0.6096491228070176
31 0.6140350877192983
32 0.618421052631579
33 0.6228070175438597
34 0.6271929824561403
35 0.631578947368421
36 0.6359649122807017
37 0.6403508771929824
38 0.6447368421052632
39 0.6491228070175439
40 0.6535087719298246


100%|██████████| 262/262 [1:03:13<00:00, 14.48s/it]

0 0.3511450381679389
1 0.4122137404580153
2 0.4618320610687023
3 0.5
4 0.5343511450381679
5 0.5648854961832062
6 0.5954198473282443
7 0.6145038167938931
8 0.6335877862595419
9 0.6526717557251909
10 0.6679389312977099
11 0.6793893129770993
12 0.6908396946564885
13 0.7022900763358778
14 0.7099236641221374
15 0.7175572519083969
16 0.7251908396946565
17 0.732824427480916
18 0.7404580152671756
19 0.7480916030534351
20 0.7557251908396947
21 0.7595419847328244
22 0.7633587786259542
23 0.767175572519084
24 0.7709923664122137
25 0.7748091603053435
26 0.7786259541984732
27 0.7824427480916031
28 0.7862595419847328
29 0.7900763358778626
30 0.7938931297709924
31 0.7977099236641222
32 0.8015267175572519
33 0.8053435114503816
34 0.8091603053435115
35 0.8129770992366412
36 0.816793893129771
37 0.8206106870229007
38 0.8244274809160306
39 0.8282442748091603
40 0.8320610687022901


In [18]:
with open('./explanations/diabtetes_anc_global_class0.pkl', 'rb') as f:
    exp_0 = pickle.load(f)

with open('./explanations/diabtetes_anc_global_class1.pkl', 'rb') as f:
    exp_1 = pickle.load(f)

In [19]:
def normal_fi(feat_imp):
    feat_imp = np.array(feat_imp) + 1e-9
    return np.abs(feat_imp) / np.sum(np.abs(feat_imp))

In [20]:
normal_feat_imp1 = normal_fi(exp1)
normal_feat_imp2 = normal_fi(exp2)

In [21]:
def global_identity(feat_imp1, feat_imp2):
    sum = 0
    for i in range(len(feat_imp1)):
        if(feat_imp1[i] == feat_imp2[i]):
            sum += 1
    return sum/len(feat_imp1)

i = global_identity(normal_feat_imp1, normal_feat_imp2)
i


0.0

In [22]:
#Entropy Ratio
Ser = np.sum(normal_feat_imp1*np.log(normal_feat_imp1))/np.log(1/len(normal_feat_imp1))

# Kullback-Leibler Divergence
Skl = np.sum(normal_feat_imp1*np.log(normal_feat_imp1/(1/len(normal_feat_imp1))))

In [23]:
def calc_gini(pfi):
    sum = 0
    for i in range(len(pfi)):
        sum_curr = 0
        for j in range(len(pfi)):
            sum_curr += np.abs(pfi[i]-pfi[j])
        sum += sum_curr
    
    return sum/(2*len(pfi)**2)*(np.sum(pfi)/len(pfi))

In [24]:
Sg = calc_gini(normal_feat_imp1)

In [25]:
Ser, Skl, Sg

(0.8835989740993055, 0.4322635983205434, 0.00030108661614839223)

In [26]:
def calc_alpha_fi(normal_pfi, alpha):
    j_inst = 0
    sum = 0
    for i in range(len(normal_pfi)-1, -1, -1):
        sum += normal_pfi[i]
        if sum<=alpha:
            j_inst = i
        else:
            break
    return 1- (j_inst/len(normal_pfi))

In [27]:
calc_alpha_fi(normal_feat_imp1, 0.8)

0.7804878048780488

In [28]:
np.linalg.norm(normal_fi(exp_0) - normal_fi(exp_1), ord=2)

0.20889363011664128